<a href="https://colab.research.google.com/github/NicolasYPS/HTML/blob/master/YOLOv4_2classes_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# YOLOv4 (Darknet) — Treinamento com 2 classes (Strawberry, Brocolli)

Notebook pronto para Google Colab, passo a passo, célula a célula.

## Pré-requisitos no seu Google Drive

Coloque em `MyDrive/TreinamentoCustomYOLO/` os arquivos:
- `obj_train.zip` → imagens + labels de treino
- `valid.zip` → imagens + labels de validação

As anotações **YOLO** (`.txt`) devem ficar no **mesmo diretório** de cada imagem, com **mesmo nome-base**.

In [7]:
# Célula 0 — Verificar GPU
import tensorflow as tf
device_name = tf.test.gpu_device_name()
print("GPU encontrada:", device_name if device_name else "Nenhuma GPU. (Ative em: Runtime > Change runtime type > GPU)")

GPU encontrada: Nenhuma GPU. (Ative em: Runtime > Change runtime type > GPU)


In [8]:
# Célula 1 — Montar Google Drive e preparar pastas
from google.colab import drive
drive.mount('/content/drive')

# Atalho para facilitar caminhos
!ln -s /content/drive/MyDrive/TreinamentoCustomYOLO /data_yolo || true
!mkdir -p /content/darknet

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
ln: failed to create symbolic link '/data_yolo/TreinamentoCustomYOLO': File exists


In [9]:
# Célula 2 — Dependências do SO e clone limpo do Darknet
%%bash
set -euo pipefail
apt-get update -qq
apt-get install -y -qq libopencv-dev

rm -rf /content/darknet
cd /content
git clone https://github.com/AlexeyAB/darknet.git
cd darknet


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Cloning into 'darknet'...


In [10]:
%%bash
set -euxo pipefail
cd /content
rm -rf darknet
git clone --depth 1 https://github.com/AlexeyAB/darknet.git
cd darknet

# 1ª tentativa: com CUDNN_HALF=1
if make -j"$(nproc)" GPU=1 CUDNN=1 CUDNN_HALF=1 OPENCV=1; then
  echo "✅ Compilado com CUDNN_HALF=1"
else
  echo "⚠️ Falhou com CUDNN_HALF=1 — tentando com CUDNN_HALF=0"
  make clean
  make -j"$(nproc)" GPU=1 CUDNN=1 CUDNN_HALF=0 OPENCV=1
fi


mkdir -p ./obj/
mkdir -p backup
mkdir -p results
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -DCUDNN_HALF -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -rdynamic -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -DCUDNN_HALF -c ./src/image_opencv.cpp -o obj/image_opencv.o
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -DCUDNN_HALF -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -rdynamic -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -DCUDNN_HALF -c ./src/http_stream.cpp -o obj/http_stream.o
gcc -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCU

+ cd /content
+ rm -rf darknet
+ git clone --depth 1 https://github.com/AlexeyAB/darknet.git
Cloning into 'darknet'...
+ cd darknet
++ nproc
+ make -j2 GPU=1 CUDNN=1 CUDNN_HALF=1 OPENCV=1
./src/http_stream.cpp: In member function ‘bool JSON_sender::write(const char*)’:
./src/http_stream.cpp:253:21: warning: unused variable ‘n’ [-Wunused-variable]
  253 |                 int n = _write(client, outputbuf, outlen);
      |                     ^
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:945:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
  945 |                 float rgb[3];
      |                       ^~~
./src/image_opencv.cpp: In function ‘void cv_draw_object(image, float*, int, int, int*, float*, int*, int, char**)’:
./src/image_opencv.cpp:1443:14: warning: unused variable ‘buff’ [-Wunused-variable]
 1443 |         char buff[100];
      |              ^~~

> Se houver erro com `sm_89`, volte nesta célula e troque `sm_89` por `sm_70` **ou** remova essa parte e rode novamente.

In [11]:
# Célula 4 — Unzip do dataset a partir do Drive
%%bash
set -euo pipefail
cd /content/darknet
unzip -oq /data_yolo/obj_train.zip -d data/
unzip -oq /data_yolo/valid.zip     -d data/
echo "Estrutura em /content/darknet/data:"
ls -la data | head -n 50

Estrutura em /content/darknet/data:
total 1848
drwxr-xr-x  5 root root   4096 Aug 22 16:07 .
drwxr-xr-x 16 root root   4096 Aug 22 16:07 ..
-rw-r--r--  1 root root 140047 Aug 22 16:05 9k.tree
-rw-r--r--  1 root root    387 Aug 22 16:05 coco9k.map
-rw-r--r--  1 root root    625 Aug 22 16:05 coco.names
-rw-r--r--  1 root root 163759 Aug 22 16:05 dog.jpg
-rw-r--r--  1 root root 141886 Aug 22 16:05 eagle.jpg
-rw-r--r--  1 root root 382965 Aug 22 16:05 giraffe.jpg
-rw-r--r--  1 root root     80 Aug 22 16:05 goal.txt
-rw-r--r--  1 root root 133495 Aug 22 16:05 horses.jpg
-rw-r--r--  1 root root 218420 Aug 22 16:05 imagenet.labels.list
-rw-r--r--  1 root root 246356 Aug 22 16:05 imagenet.shortnames.list
drwxr-xr-x  2 root root  20480 Aug 22 16:05 labels
drwxr-xr-x  3 root root  81920 Aug 19 15:31 obj_train
-rw-r--r--  1 root root   5476 Aug 22 16:05 openimages.names
-rw-r--r--  1 root root 113880 Aug 22 16:05 person.jpg
-rw-r--r--  1 root root 174515 Aug 22 16:05 scream.jpg
drwxr-xr-x  3 root

In [12]:
# Célula 5 — Criar obj.names (2 classes) e obj.data
%%bash
set -euo pipefail
cd /content/darknet
cat > data/obj.names <<'EOF'
Strawberry
Brocolli
EOF

cat > data/obj.data <<'EOF'
classes = 2
train   = data/train.txt
valid   = data/valid.txt
names   = data/obj.names
backup  = /content/darknet/backup
EOF

echo "data/obj.names:"; cat data/obj.names
echo "----"
echo "data/obj.data:"; cat data/obj.data


data/obj.names:
Strawberry
Brocolli
----
data/obj.data:
classes = 2
train   = data/train.txt
valid   = data/valid.txt
names   = data/obj.names
backup  = /content/darknet/backup


In [13]:
# Célula 6 — Gerar train.txt e valid.txt
%%bash
set -euo pipefail
cd /content/darknet
rm -f data/train.txt data/valid.txt
find /content/darknet/data/obj_train -type f \( -iname '*.jpg' -o -iname '*.png' -o -iname '*.jpeg' \) | sort > data/train.txt
find /content/darknet/data/valid     -type f \( -iname '*.jpg' -o -iname '*.png' -o -iname '*.jpeg' \) | sort > data/valid.txt
echo "train:" $(wc -l < data/train.txt) "— valid:" $(wc -l < data/valid.txt)
head -n 3 data/train.txt; echo "..."; tail -n 3 data/train.txt

train: 975 — valid: 200
/content/darknet/data/obj_train/00030b70a3abfcea.jpg
/content/darknet/data/obj_train/000e1654f1d9b2cb.jpg
/content/darknet/data/obj_train/00164cfbc5281399.jpg
...
/content/darknet/data/obj_train/fe9965fdff48e2b2.jpg
/content/darknet/data/obj_train/feb0236c0b6281a9.jpg
/content/darknet/data/obj_train/ffc2a16108591b7e.jpg


In [14]:
# Célula 7 — Preparar cfg (yolov4_custom.cfg) com classes=2 e filters=21
from pathlib import Path
import re
cfg_src = Path('/content/darknet/cfg/yolov4-custom.cfg')
cfg_dst = Path('/content/darknet/cfg/yolov4_custom.cfg')

cfg_dst.write_text(cfg_src.read_text())
text = cfg_dst.read_text()

# ajustes básicos
text = text.replace('\nbatch=64\n', '\nbatch=64\n')  # mantém
text = text.replace('\nsubdivisions=16\n', '\nsubdivisions=32\n')
text = text.replace('\nwidth=416\n', '\nwidth=608\n')
text = text.replace('\nheight=416\n', '\nheight=608\n')

# max_batches/steps
text = re.sub(r'^max_batches=.*$', 'max_batches=6000', text, flags=re.MULTILINE)
text = re.sub(r'^steps=.*$', 'steps=4800,5400', text, flags=re.MULTILINE)

# classes nas seções [yolo]
text = re.sub(r'(?m)^(classes=)\d+', r'\g<1>2', text)

# filters=21 imediatamente antes de cada [yolo]
lines = text.splitlines()
yolo_idx = [i for i,l in enumerate(lines) if l.strip()=='[yolo]']
for yi in yolo_idx:
    for j in range(yi-1, -1, -1):
        if lines[j].strip().startswith('filters='):
            lines[j] = 'filters=21'
            break
text = '\n'.join(lines)
cfg_dst.write_text(text)
print('CFG pronto em:', cfg_dst)

# Conferência
!nl -ba /content/darknet/cfg/yolov4_custom.cfg | grep -n "\[yolo\]\|\bfilters=\|\bclasses="

CFG pronto em: /content/darknet/cfg/yolov4_custom.cfg
32:    32	filters=32
42:    42	filters=64
50:    50	filters=64
61:    61	filters=64
69:    69	filters=32
77:    77	filters=64
89:    89	filters=64
100:   100	filters=64
110:   110	filters=128
118:   118	filters=64
129:   129	filters=64
137:   137	filters=64
145:   145	filters=64
157:   157	filters=64
165:   165	filters=64
177:   177	filters=64
188:   188	filters=128
198:   198	filters=256
206:   206	filters=128
217:   217	filters=128
225:   225	filters=128
233:   233	filters=128
245:   245	filters=128
253:   253	filters=128
265:   265	filters=128
273:   273	filters=128
285:   285	filters=128
293:   293	filters=128
306:   306	filters=128
314:   314	filters=128
326:   326	filters=128
334:   334	filters=128
346:   346	filters=128
354:   354	filters=128
366:   366	filters=128
374:   374	filters=128
386:   386	filters=128
397:   397	filters=256
407:   407	filters=512
415:   415	filters=256
426:   426	filters=256
434:   434	filters=256
44

In [15]:
# Célula 8 — Pesos pré-treinados e garantia de build
%%bash
set -euo pipefail
cd /content/darknet
wget -q https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.conv.137 -O yolov4.conv.137
make -j"$(nproc)"

chmod +x *.sh


In [17]:
# Célula 9 — Treinamento (com log + mAP)
%%bash
set -euo pipefail
cd /content/darknet
./darknet detector train data/obj.data cfg/yolov4_custom.cfg yolov4.conv.137 -dont_show -map 2>&1

./darknet: error while loading shared libraries: libcuda.so.1: cannot open shared object file: No such file or directory


CalledProcessError: Command 'b'set -euo pipefail\ncd /content/darknet\n./darknet detector train data/obj.data cfg/yolov4_custom.cfg yolov4.conv.137 -dont_show -map 2>&1 \n'' returned non-zero exit status 127.

In [ ]:
# Célula 10 — Utilitários de visualização (loss/mAP + chart)
import os, re, numpy as np, matplotlib.pyplot as plt, cv2

def mostrar(caminho):
    img = cv2.imread(caminho)
    if img is None:
        raise FileNotFoundError(caminho)
    plt.figure(figsize=(18,10)); plt.axis('off')
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.show()

def mostrar_chart():
    for c in ["chart.png","results/chart.png","backup/chart.png","map.png","mAP.png"]:
        if os.path.exists(c):
            print("Exibindo:", c); mostrar(c); return c
    print("Nenhum chart encontrado."); return None

def _parse_darknet_log(log="train.log"):
    iters, loss, it_map, mAP = [], [], [], []
    if not os.path.exists(log):
        return np.array([]),np.array([]),np.array([]),np.array([])
    with open(log, "r", errors="ignore") as f:
        for line in f:
            m = re.search(r"(\d+):.*?([0-9]*\.?[0-9]+)\s+avg loss", line)
            if m:
                iters.append(int(m.group(1)))
                loss.append(float(m.group(2)))
            m2 = re.search(r"map\s*=\s*([0-9]*\.?[0-9]+)", line)
            if m2 and iters:
                it_map.append(iters[-1])
                mAP.append(float(m2.group(1)))
    return np.array(iters), np.array(loss), np.array(it_map), np.array(mAP)

def plot_loss_map(log="train.log"):
    it, ls, it_m, mp = _parse_darknet_log(log)
    if it.size:
        plt.figure(); plt.plot(it, ls); plt.grid(True)
        plt.title("YOLOv4 — Avg loss"); plt.xlabel("Iterações"); plt.ylabel("Avg loss"); plt.show()
    else:
        print("Sem 'avg loss' no log.")
    if it_m.size:
        plt.figure(); plt.plot(it_m, mp); plt.grid(True)
        plt.title("Validação — mAP (%)"); plt.xlabel("Iterações"); plt.ylabel("mAP (%)"); plt.show()
    else:
        print("Sem 'mAP' no log (use -map no treino).")

_ = mostrar_chart()
plot_loss_map("train.log")

In [ ]:
# Célula 11 — Inferência de teste (gera predictions.jpg)
%%bash
set -euo pipefail
cd /content/darknet
TEST_IMG=$(head -n 1 data/valid.txt)
echo "Testando em: $TEST_IMG"
WEIGHTS=$(ls -1 backup/*.weights 2>/dev/null | grep -E "best|final" | head -n 1)
echo "Pesos: $WEIGHTS"
./darknet detector test data/obj.data cfg/yolov4_custom.cfg "$WEIGHTS" "$TEST_IMG" -thresh 0.25 -dont_show -ext_output || true

In [ ]:
from IPython.display import Image, display
from pathlib import Path
p = Path('/content/darknet/predictions.jpg')
display(Image(filename=str(p))) if p.exists() else print('predictions.jpg não gerado.')

In [ ]:
# Célula 12 — Salvar artefatos no Drive (pesos/log/gráficos)
%%bash
set -euo pipefail
cd /content/darknet
mkdir -p /data_yolo/treino_resultados
cp -f backup/*.weights train.log chart.png 2>/dev/null /data_yolo/treino_resultados || true
echo "Copiados para: /data_yolo/treino_resultados"
ls -lh /data_yolo/treino_resultados || true

### Dicas e solução de problemas
- **Erro `nvcc fatal : A single input file ...`** → `ARCH` mal definido. Use a **Célula 3** exatamente como está.
- **Erro com `sm_89`** → Remova `-gencode ... sm_89` ou troque por `sm_70` e recompile.
- **Sem `mAP` no log** → garanta o parâmetro `-map` no treino (Célula 9).
- **CUDA OOM (VRAM insuficiente)** → aumente `subdivisions` (ex.: 64) ou reduza `width/height` para `512` no CFG.
- **`filters` incorretos** → com 2 classes, antes de cada `[yolo]` deve ser `filters=21`. A **Célula 7** faz isso automaticamente.
- **Listas vazias** (`train.txt`/`valid.txt`) → refaça Células 5 e 6 e confira caminhos.